In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [2]:
df1 = pd.read_csv("./youtube_audio(Angry).csv")
df2 = pd.read_csv("./youtube_audio(Bright).csv")
df3 = pd.read_csv("./youtube_audio(Dark).csv")
df4 = pd.read_csv("./youtube_audio(Happy).csv")
df5 = pd.read_csv("./youtube_audio(Sad).csv")

In [3]:
path_dir

'C:\\Users\\gmlrn\\Desktop\\AI_SCHOOL\\기업 프로젝트\\youtube_audio_long'

In [2]:
import os

path_dir = %pwd

file_list = os.listdir(path_dir + '\split_audio_44100')

len(file_list)

23849

In [3]:
last_df = pd.read_csv("./real_last.csv")
last_df = last_df.drop(["Unnamed: 0"], 1)
df_train = last_df.sample(frac=1)
df_train

,title,mood
17634,Slim To None_split_5.wav,밝게
11320,Life is Tough_split_10.wav,밝게
15707,Red Ant_split_30.wav,밝게
1619,Barilicious_split_12.wav,화남
19212,Take it Back_split_15.wav,밝게
...,...,...
17729,So Long Analog_split_5.wav,밝게
15143,Pluck It Up_split_10.wav,밝게
21405,Twitch_split_15.wav,밝게
11728,Lost Highway_split_29.wav,슬픔


In [119]:
df5["title"].value_counts()

Requiem In Cello     1
Digital Solitude     1
We'll Meet Again     1
Heartache            1
Lost Highway         1
                    ..
The Hungry Ghost     1
Sao Meo              1
Earnest              1
I'll Remember You    1
Disconcerned         1
Name: title, Length: 136, dtype: int64

In [82]:
df3["title"] = df3["title"].apply(lambda x : x+".wav")


In [87]:
a = df3["title"].tolist()

set(a) - set(file_list)


{'A Great Darkness Approaches, Can You Feel It?.wav'}

In [125]:
df_train = pd.concat( [df1, df2, df3, df4, df5] )
df_train = df_train.reset_index(drop=True)
df_train = df_train[ ["title", "mood"] ].sample(frac = 1).reset_index(drop=True)
df_train["title"] = df_train["title"].apply(lambda x : x+".wav")

In [6]:
df_train

,title,mood
18198,Spooky SZN_split_16.wav,어둡게
19932,The New Darker You_split_25.wav,어둡게
22148,Where The Trap Is_split_25.wav,어둡게
7181,Gallato_split_17.wav,어둡게
915,Almost August_split_30.wav,행복
...,...,...
19914,The New Darker You_split_7.wav,어둡게
15209,Pop Goes The Weasel_split_4.wav,행복
3922,Crystal Towers of the Moon_split_18.wav,어둡게
12203,Manea 2_split_7.wav,밝게


In [4]:
df_train.mood.value_counts()

어둡게    5368
행복     5205
밝게     5155
화남     3718
슬픔     3510
Name: mood, dtype: int64

In [5]:
df_train["mood"].value_counts()

어둡게    5368
행복     5205
밝게     5155
화남     3718
슬픔     3510
Name: mood, dtype: int64

In [4]:
labels = set(df_train.mood.to_list())
labels

{'밝게', '슬픔', '어둡게', '행복', '화남'}

In [5]:
label2int = {l:i for i, l in enumerate(labels)}

In [6]:
#2020-11-11 일자
label2int

{'어둡게': 0, '슬픔': 1, '화남': 2, '행복': 3, '밝게': 4}

In [7]:
plain_y_train = np.array([label2int[l] for l in df_train.mood])

In [8]:
path = ["./split_audio_44100/"+fname for fname in df_train.title ]

sampling_rate = 44100
duration = 1
hop_length = 347 # to make time steps 128
fmin = 20
fmax = sampling_rate // 2
n_mels = 128 #128
n_fft = n_mels * 20

random_state = 42
test_size = 0.01

samples_per_file = 5

dims = (128, 128, 1) #128,128,1
samples = sampling_rate * duration

In [9]:
import librosa
import librosa.display

print(librosa.__version__)

0.8.0


In [10]:
def audio_to_melspectrogram(audio):
    spectrogram = librosa.feature.melspectrogram(audio,
                                                 sr=sampling_rate,
                                                 n_mels=n_mels,
                                                 hop_length=hop_length,
                                                 n_fft=n_fft,
                                                 fmin=fmin,
                                                 fmax=fmax)
    spectrogram = librosa.power_to_db(spectrogram)
    spectrogram = spectrogram.astype(np.float32)
    return spectrogram

In [11]:
def read_audio(pathname, trim_long_data):
    y, sr = librosa.load(pathname, sr=sampling_rate)
    # trim silence
    if 0 < len(y): # workaround: 0 length causes error
        y, _ = librosa.effects.trim(y) # trim, top_db=default(60)
    # make it unified length to conf.samples
    if len(y) > samples: # long enough
        if trim_long_data:
            y = y[0:0+samples]
    else: # pad blank
        padding = samples - len(y)    # add padding at both ends
        offset = padding // 2
        y = np.pad(y, (offset, samples - len(y) - offset), 'constant')
    return y

In [12]:
def read_as_melspectrogram(pathname, trim_long_data, debug_display=False):
    x = read_audio(pathname, trim_long_data)
    mels = audio_to_melspectrogram(x)
    if debug_display:
        IPython.display.display(IPython.display.Audio(x, rate=conf.sampling_rate))
        show_melspectrogram(conf, mels)
    return mels

In [13]:
def mels_len(mels):
    """Gets lenfth of log mel-spectrogram."""
    return mels.shape[1]

In [14]:
def mels_populate_samples(org_mels, targ_len, targ_samples):
    """Populates training samples from original full length wave's log mel-spectrogram."""
    org_len = mels_len(org_mels)
    assert org_len >= targ_len
    one_step = np.min([float(targ_len),
                       (org_len - targ_len + 1) / targ_samples])
    generated = []
    for i in range(targ_samples):
        cur = int(one_step * i)
        generated.append(org_mels[:, cur:cur+targ_len])
    return np.array(generated)

In [15]:
def flatten_y_if_onehot(y):
    """De-one-hot y, i.e. [0,1,0,0,...] to 1 for all y."""
    return y if len(np.array(y).shape) == 1 else np.argmax(y, axis = -1)

In [16]:
def mels_demux_XX_y(XX, y):
    """De-multiplex data."""
    dims = XX.shape
    X = XX.reshape(dims[0] * dims[1], dims[2], dims[3], dims[4])
    y = np.repeat(y, dims[1], axis=0) # ex. if dims[1]==2) [1, 2, 3] -> [1, 1, 2, 2, 3, 3]
    return X, y

In [17]:
def train_valid_split_multiplexed(XX, y, demux=True, delta_random_state=0):
    """Splits multiplexed set of data.
    
    XX[sample_no, multiplex_no, ...] to be 3 or more dimentional data vector.
    y[sample_no] can be both one-hot or label index.
    demux==True will output X, False will output raw split XX"""
    assert len(XX) == len(y)
    def split_fold(XX, y, fold_list, demux):
        _XX, _y = XX[fold_list], y[fold_list]
        if demux:
            return mels_demux_XX_y(_XX, _y)
        else:
            return _XX, _y
    # decode y if it is one-hot vector
    _y = flatten_y_if_onehot(y)
    print()
    # split train/valid
    train_fold, valid_fold, _, _ = train_test_split(list(range(len(_y))), _y,
                             test_size=0.3,
                             random_state=random_state + delta_random_state)
    X_or_XX_train, y_train = split_fold(XX, y, train_fold, demux)
    X_or_XX_valid, y_valid = split_fold(XX, y, valid_fold, demux)
    # train/valid are ready
    return X_or_XX_train, y_train, X_or_XX_valid, y_valid

In [18]:
samples_per_file, dims

(5, (128, 128, 1))

In [28]:
# !pip install librosa==0.8.0

Processing c:\users\gmlrn\appdata\local\pip\cache\wheels\32\2c\ce\86e49d4769aceba728421c24c0d726054bf4ca01175ff42bdd\librosa-0.8.0-py3-none-any.whl
  Attempting uninstall: librosa
    Found existing installation: librosa 0.6.0
    Uninstalling librosa-0.6.0:
      Successfully uninstalled librosa-0.6.0


In [19]:
# 실행
XX = np.zeros( ( len( path ),  samples_per_file, dims[0], dims[1]) )
for i, file in enumerate(path):
    print(i, file)
    whole_mels = read_as_melspectrogram(file, trim_long_data=False)
    multiplexed = mels_populate_samples(whole_mels, dims[1], samples_per_file)
    XX[i, :, :, :] = multiplexed
XX = XX[..., np.newaxis]

0 ./split_audio_44100/Slim To None_split_5.wav
1 ./split_audio_44100/Life is Tough_split_10.wav
2 ./split_audio_44100/Red Ant_split_30.wav
3 ./split_audio_44100/Barilicious_split_12.wav
4 ./split_audio_44100/Take it Back_split_15.wav
5 ./split_audio_44100/Barge_split_7.wav
6 ./split_audio_44100/Ritual Cathedral_split_5.wav
7 ./split_audio_44100/Walkout_split_2.wav
8 ./split_audio_44100/Jigsaw Puzzle_split_4.wav
9 ./split_audio_44100/Jeremiah's Song_split_22.wav
10 ./split_audio_44100/Live Worms_split_17.wav
11 ./split_audio_44100/Fuzzy and True_split_16.wav
12 ./split_audio_44100/Blow Thing_split_6.wav
13 ./split_audio_44100/Beartooth Strangleheart_split_22.wav
14 ./split_audio_44100/Berlin at Night_split_13.wav
15 ./split_audio_44100/Somber_split_20.wav
16 ./split_audio_44100/Stealer_split_3.wav
17 ./split_audio_44100/Dark Thoughts_split_26.wav
18 ./split_audio_44100/Big Top_split_9.wav
19 ./split_audio_44100/Winds of Spring_split_23.wav
20 ./split_audio_44100/The Closest Without Goin

168 ./split_audio_44100/All Hail the Queen_split_20.wav
169 ./split_audio_44100/Scarab_split_17.wav
170 ./split_audio_44100/Bug Catching_split_9.wav
171 ./split_audio_44100/Parkside_split_18.wav
172 ./split_audio_44100/Pentagram_split_2.wav
173 ./split_audio_44100/Quarter Mix_split_12.wav
174 ./split_audio_44100/Digital Solitude_split_6.wav
175 ./split_audio_44100/Rockville_split_3.wav
176 ./split_audio_44100/Simon's Song_split_9.wav
177 ./split_audio_44100/Dude_split_7.wav
178 ./split_audio_44100/City of Jewels_split_18.wav
179 ./split_audio_44100/The Fiery Furnace_split_4.wav
180 ./split_audio_44100/The Missing 11th_split_16.wav
181 ./split_audio_44100/Prime_split_29.wav
182 ./split_audio_44100/Boy With A Pipe_split_24.wav
183 ./split_audio_44100/Azure_split_27.wav
184 ./split_audio_44100/My Achy Heart_split_26.wav
185 ./split_audio_44100/555_split_15.wav
186 ./split_audio_44100/Joker's Lair_split_27.wav
187 ./split_audio_44100/Tiptoe Out the Back_split_9.wav
188 ./split_audio_44100/

337 ./split_audio_44100/Pale Rider Blues_split_2.wav
338 ./split_audio_44100/Skanada_split_33.wav
339 ./split_audio_44100/Luna's Soul_split_15.wav
340 ./split_audio_44100/Tight Perm_split_12.wav
341 ./split_audio_44100/Sewer_split_26.wav
342 ./split_audio_44100/A Hand In The Dark_split_5.wav
343 ./split_audio_44100/Life After Death_split_22.wav
344 ./split_audio_44100/Keep It 98 Plus 2_split_13.wav
345 ./split_audio_44100/Monroe_split_21.wav
346 ./split_audio_44100/Colorless Aura_split_21.wav
347 ./split_audio_44100/Aletheia (Unforgetting)_split_17.wav
348 ./split_audio_44100/5-O-July_split_11.wav
349 ./split_audio_44100/Glass_split_16.wav
350 ./split_audio_44100/Yo Picasso_split_11.wav
351 ./split_audio_44100/Tripolar_split_18.wav
352 ./split_audio_44100/The Dismal Hand_split_30.wav
353 ./split_audio_44100/Sunset Riddim_split_13.wav
354 ./split_audio_44100/Tomb Raider_split_9.wav
355 ./split_audio_44100/Spring Forward_split_8.wav
356 ./split_audio_44100/Milos_split_18.wav
357 ./split_

501 ./split_audio_44100/Realization of a New Earth_split_24.wav
502 ./split_audio_44100/Bug Catching_split_17.wav
503 ./split_audio_44100/Oceans_split_4.wav
504 ./split_audio_44100/The Schooner The Better_split_6.wav
505 ./split_audio_44100/Cartoon Battle - Vadodara_split_18.wav
506 ./split_audio_44100/Key to Your Heart_split_4.wav
507 ./split_audio_44100/AnalogueCabin_split_13.wav
508 ./split_audio_44100/Black Bird - Primal Drive_split_13.wav
509 ./split_audio_44100/Lost Highway_split_11.wav
510 ./split_audio_44100/Deserts_split_2.wav
511 ./split_audio_44100/Introspective Spacewalk_split_22.wav
512 ./split_audio_44100/Jon's On Fire_split_12.wav
513 ./split_audio_44100/Cupid's Tubes_split_18.wav
514 ./split_audio_44100/My Town Yo Town_split_13.wav
515 ./split_audio_44100/Torture_split_1.wav
516 ./split_audio_44100/Ha_split_4.wav
517 ./split_audio_44100/No.8 Requiem_split_23.wav
518 ./split_audio_44100/Mistake_split_16.wav
519 ./split_audio_44100/Heartbeat of the Hood_split_2.wav
520 ./

668 ./split_audio_44100/Poe's_split_6.wav
669 ./split_audio_44100/Harlequin_split_2.wav
670 ./split_audio_44100/Festivities in Belize_split_7.wav
671 ./split_audio_44100/Lover's Stripes_split_15.wav
672 ./split_audio_44100/Gravel Road_split_2.wav
673 ./split_audio_44100/Sulking_split_8.wav
674 ./split_audio_44100/Aurora Currents_split_21.wav
675 ./split_audio_44100/Sizzle_split_14.wav
676 ./split_audio_44100/Invitation to the Castle Ball_split_21.wav
677 ./split_audio_44100/Sunset Riddim_split_11.wav
678 ./split_audio_44100/Tuff Data_split_23.wav
679 ./split_audio_44100/Missive_split_6.wav
680 ./split_audio_44100/AnalogueCabin_split_16.wav
681 ./split_audio_44100/Night Shifter_split_16.wav
682 ./split_audio_44100/Dreaming Of A New Day_split_26.wav
683 ./split_audio_44100/Friday Fugue_split_17.wav
684 ./split_audio_44100/After Thought_split_13.wav
685 ./split_audio_44100/Harpsichord Fugue_split_11.wav
686 ./split_audio_44100/Puffer_split_16.wav
687 ./split_audio_44100/Story Of A Toy_spl

...

22312 ./split_audio_44100/Long Road Ahead_split_12.wav
22313 ./split_audio_44100/Transmission_split_10.wav
22314 ./split_audio_44100/Heavy Hunter_split_6.wav
22315 ./split_audio_44100/Heartbeat of the Hood_split_9.wav
22316 ./split_audio_44100/Engine Fire_split_1.wav
22317 ./split_audio_44100/Introspective Spacewalk_split_11.wav
22318 ./split_audio_44100/Somber_split_16.wav
22319 ./split_audio_44100/Hammock Style_split_9.wav
22320 ./split_audio_44100/Renegade Jubilee_split_1.wav
22321 ./split_audio_44100/Palm Avenue_split_20.wav
22322 ./split_audio_44100/You Can't Fail_split_26.wav
22323 ./split_audio_44100/Away_split_12.wav
22324 ./split_audio_44100/Finals_split_13.wav
22325 ./split_audio_44100/Rage_split_10.wav
22326 ./split_audio_44100/Museums_split_11.wav
22327 ./split_audio_44100/Hillbilly Hologram_split_9.wav
22328 ./split_audio_44100/Happy Birthday Old Timey_split_29.wav
22329 ./split_audio_44100/Canada_split_17.wav
22330 ./split_audio_44100/Lyra Mortis_split_16.wav
22331 ./spli

22469 ./split_audio_44100/Loud In The Cloud_split_23.wav
22470 ./split_audio_44100/Red Chair_split_3.wav
22471 ./split_audio_44100/Gornin_split_15.wav
22472 ./split_audio_44100/Second Chance_split_10.wav
22473 ./split_audio_44100/Dance of the U-boat_split_9.wav
22474 ./split_audio_44100/Taking a Beating_split_8.wav
22475 ./split_audio_44100/The Missing 11th_split_18.wav
22476 ./split_audio_44100/Hillbilly Hologram_split_4.wav
22477 ./split_audio_44100/My Town Yo Town_split_11.wav
22478 ./split_audio_44100/Scanline_split_24.wav
22479 ./split_audio_44100/Yucatan Peninsula_split_6.wav
22480 ./split_audio_44100/Happy Birthday Soul_split_19.wav
22481 ./split_audio_44100/Natural_split_13.wav
22482 ./split_audio_44100/Bringin' It Back_split_18.wav
22483 ./split_audio_44100/Apolllo_split_24.wav
22484 ./split_audio_44100/Taps Bugle Call_split_10.wav
22485 ./split_audio_44100/Observer_split_23.wav
22486 ./split_audio_44100/Cooper Ave_split_18.wav
22487 ./split_audio_44100/Don't Shoot!_split_12.w

22626 ./split_audio_44100/Fanta Mankane_split_24.wav
22627 ./split_audio_44100/Secret Second Earth_split_29.wav
22628 ./split_audio_44100/Rainbow Forest_split_14.wav
22629 ./split_audio_44100/Positioned_split_22.wav
22630 ./split_audio_44100/Big Top_split_20.wav
22631 ./split_audio_44100/Winds of Spring_split_22.wav
22632 ./split_audio_44100/Cooper Ave_split_7.wav
22633 ./split_audio_44100/Time Is Forever_split_25.wav
22634 ./split_audio_44100/Erase Me_split_7.wav
22635 ./split_audio_44100/If I Had a Chicken_split_18.wav
22636 ./split_audio_44100/Simon's Song_split_24.wav
22637 ./split_audio_44100/South Street Strut_split_13.wav
22638 ./split_audio_44100/Wok Hard_split_5.wav
22639 ./split_audio_44100/Perihelion_split_3.wav
22640 ./split_audio_44100/Black Bird - Primal Drive_split_9.wav
22641 ./split_audio_44100/Greengrass_split_13.wav
22642 ./split_audio_44100/Merry Go Round_split_7.wav
22643 ./split_audio_44100/Organic Apples_split_1.wav
22644 ./split_audio_44100/Mr. Sunny Face_split_

22782 ./split_audio_44100/Marockatoo_split_27.wav
22783 ./split_audio_44100/Space Racer_split_6.wav
22784 ./split_audio_44100/Nightingale_split_19.wav
22785 ./split_audio_44100/Stopping By the Inn_split_13.wav
22786 ./split_audio_44100/Luke's Rage_split_10.wav
22787 ./split_audio_44100/Friday Fugue_split_5.wav
22788 ./split_audio_44100/Ego Push_split_4.wav
22789 ./split_audio_44100/80's Video Game Death_split_9.wav
22790 ./split_audio_44100/I'll Remember You_split_19.wav
22791 ./split_audio_44100/Goddess of War_split_15.wav
22792 ./split_audio_44100/Revolutionary_split_34.wav
22793 ./split_audio_44100/Krishna's Calliope_split_30.wav
22794 ./split_audio_44100/Fresno Alley_split_13.wav
22795 ./split_audio_44100/Rain Cloud Music_split_18.wav
22796 ./split_audio_44100/Scanner_split_1.wav
22797 ./split_audio_44100/Fast Anxiety_split_23.wav
22798 ./split_audio_44100/Shadowing_split_22.wav
22799 ./split_audio_44100/Uh Oh_split_8.wav
22800 ./split_audio_44100/The Snowbird Strut_split_18.wav
22

22939 ./split_audio_44100/Baby We Did It_split_9.wav
22940 ./split_audio_44100/Lovable Clown Sit Com_split_15.wav
22941 ./split_audio_44100/Groove_split_4.wav
22942 ./split_audio_44100/Pluck It Up_split_2.wav
22943 ./split_audio_44100/Take Me to the Depths_split_21.wav
22944 ./split_audio_44100/Colorless Aura_split_28.wav
22945 ./split_audio_44100/Grut_split_5.wav
22946 ./split_audio_44100/Robot Boogie_split_19.wav
22947 ./split_audio_44100/Alaska Sky_split_5.wav
22948 ./split_audio_44100/Boardroom Theme_split_2.wav
22949 ./split_audio_44100/Dark Toys_split_34.wav
22950 ./split_audio_44100/Contact_split_3.wav
22951 ./split_audio_44100/So Long Analog_split_5.wav
22952 ./split_audio_44100/Pluck It Up_split_10.wav
22953 ./split_audio_44100/Twitch_split_15.wav
22954 ./split_audio_44100/Lost Highway_split_29.wav
22955 ./split_audio_44100/Honey and Milk_split_27.wav


In [21]:
XX.shape

(22956, 5, 128, 128, 1)

In [22]:
X_train, y_train, X_test, y_test = train_valid_split_multiplexed(XX, plain_y_train, demux=True)
np.save("X_train_44100.npy", X_train)
np.save("y_train_44100.npy", y_train)
np.save("X_test_44100.npy", X_test)
np.save("y_test_44100.npy", y_test)

In [23]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((80345, 128, 128, 1), (80345,), (34435, 128, 128, 1), (34435,))

In [8]:
y_test = np.load("./y_test.npy")

In [9]:
# print(X_train.shape)
# print(X_test.shape)
# print(y_train.shape)
print(y_test.shape)

(34435,)


X_train (80345, 128, 128, 1)<br>
X_test  (34435, 128, 128, 1)<br>
y_train (80345,)<br>
y_test  (34435,)


In [48]:
print(X_train[:40000,...].shape)

(40000, 128, 128, 1)


In [17]:
train = 80345
test = 34435

In [18]:
n = 10
train_n = train // n
test_n = test // n

print(train_n, test_n)

8034 3443


In [37]:
for i in range(1, n):
    
    idx = "_"+str(i)
    
#     print("Train :", X_train[train_n * (i-1):train_n * i, ...].shape)
    print("X_train"+idx+".npy")
    np.save("X_train"+idx+".npy", X_train[train_n * (i-1):train_n * i, ...])
    
#     print("Test :", X_test[test_n * (i-1):test_n * i, ...].shape)
    print("X_test"+idx+".npy")
    np.save("X_test"+idx+".npy", X_test[test_n * (i-1):test_n * i, ...])
    
#     print("Train :", y_train[train_n * (i-1):train_n * i, ...].shape)
    print("y_train"+idx+".npy")
    np.save("y_train"+idx+".npy", y_train[train_n * (i-1):train_n * i, ...])
    
#     print("Test :", y_test[test_n * (i-1):test_n * i, ...].shape)
    print("y_test"+idx+".npy")
    np.save("y_test"+idx+".npy", y_test[test_n * (i-1):test_n * i, ...])
    

    
# print("Train :", X_train[train_n * 9:, ...].shape)
# print("Test :", X_test[test_n * 9:, ...].shape)

print("X_train"+"10"+".npy")
np.save("X_train"+"10"+".npy", X_train[train_n * 9:, ...])

print("X_test"+"10"+".npy")
np.save("X_test"+"10"+".npy", X_test[test_n * 9:, ...])

print("y_train"+"10"+".npy")
np.save("y_train"+"10"+".npy", y_train[train_n * 9:, ...])

print("y_test"+"10"+".npy")
np.save("y_test"+"10"+".npy", y_test[test_n * 9:, ...])

X_train_1.npy
X_test_1.npy
y_train_1.npy
y_test_1.npy
X_train_2.npy
X_test_2.npy
y_train_2.npy
y_test_2.npy
X_train_3.npy
X_test_3.npy
y_train_3.npy
y_test_3.npy
X_train_4.npy
X_test_4.npy
y_train_4.npy
y_test_4.npy
X_train_5.npy
X_test_5.npy
y_train_5.npy
y_test_5.npy
X_train_6.npy
X_test_6.npy
y_train_6.npy
y_test_6.npy
X_train_7.npy
X_test_7.npy
y_train_7.npy
y_test_7.npy
X_train_8.npy
X_test_8.npy
y_train_8.npy
y_test_8.npy
X_train_9.npy
X_test_9.npy
y_train_9.npy
y_test_9.npy
X_train10.npy
X_test10.npy
y_train10.npy
y_test10.npy


In [54]:
X_train_1 = X_train[:25000, ...]
y_train_1 = y_train[:25000, ...]
X_test_1 = X_test[:5000, ...]
y_test_1 = y_test[:5000, ...]

X_train_2 = X_train[40001:, ...]
y_train_2 = y_train[40001:, ...]
X_test_2 = X_test[15001:, ...]
y_test_2 = y_test[15001:, ...]

In [55]:
np.save("X_train_3.npy", X_train_1)
np.save("y_train_3.npy", y_train_1)
np.save("X_test_3.npy", X_test_1)
np.save("y_test_3.npy", y_test_1)

# np.save("X_train_2.npy", X_train_2)
# np.save("y_train_2.npy", y_train_2)
# np.save("X_test_2.npy", X_test_2)
# np.save("y_test_2.npy", y_test_2)

In [22]:
from collections import Counter

In [23]:
Counter(y_train)

Counter({1: 18725, 3: 18215, 0: 17945, 4: 12980, 2: 12480})

In [1]:
import torch
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1650 Ti


In [44]:
X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

In [45]:
X_train.shape

torch.Size([80345, 128, 128, 1])